### Using Someshar's work and updating few entries

In [122]:
import pandas as pd
import numpy as np
import numpy as np
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection as xval
from sklearn.datasets import fetch_openml
import forestci as fci
from sklearn import metrics
from sklearn.metrics import r2_score
import statistics
import matplotlib.pyplot as plt

Failed to import duecredit due to No module named 'duecredit'


In [17]:
df = pd.read_csv('post_impute_environmental_43_markets_2008_2018.csv')
df= df.drop(['Unnamed: 0'], axis=1)
df.head()

,Market,Arrivals,Date,Modal_Price,District,Imp_Price,Imp_Arrival,PriceNanFlag,ArrivalNanFlag,latitude,...,Soil_Type,Humidity,Sunlight,Temperature,Avg_Precipitation,Last_Precipitation,Year,Month_name,Day,Month
0,Agra,NaN,01-Jan-2008,NaN,Agra,533.577743,136.448320,True,True,27.25,...,1.999985,53.146736,1.265486e+07,289.903276,3.72529,3.72529,2008,Jan,1,1
1,Agra,NaN,02-Jan-2008,NaN,Agra,564.358912,141.135159,True,True,27.25,...,1.999985,55.914157,1.211169e+07,289.533809,3.72529,3.72529,2008,Jan,2,1
2,Agra,NaN,03-Jan-2008,NaN,Agra,562.825324,145.897888,True,True,27.25,...,1.999985,61.595423,1.274031e+07,288.712016,3.72529,3.72529,2008,Jan,3,1
3,Agra,NaN,04-Jan-2008,NaN,Agra,560.865227,220.000000,True,True,27.25,...,1.999985,59.810162,1.259190e+07,289.971033,3.72529,3.72529,2008,Jan,4,1
4,Agra,NaN,05-Jan-2008,NaN,Agra,538.705688,218.330750,True,True,27.25,...,1.999985,55.464904,1.240704e+07,291.377191,3.72529,3.72529,2008,Jan,5,1


In [26]:
def yearly_nan_count():
    yearly_nan ={}
    years= df['Year'].unique()
    for year in years:
        yearly_nan[year]=0

    for i in range(df.shape[0]):
        if df['PriceNanFlag'][i]:
            year = df['Year'][i]
            yearly_nan[year]+=1

    print(yearly_nan)
    



In [28]:
yearly_nan_count()

{2008: 5196, 2009: 5086, 2010: 4324, 2011: 4058, 2012: 4116, 2013: 4983, 2014: 4651, 2015: 4227, 2016: 5435, 2017: 3976, 2018: 2789}


In [32]:
def missing_freq_count():
    missing_count = [i for i in range(1000)]

    missing_count_dict ={}

    for i in missing_count:
        missing_count_dict[i]=0

    flag=False    
    for i in range(df.shape[0]):

        if df['PriceNanFlag'][i] == True:
            if flag:
                count+=1
            else:
                count=1
                flag=True

        if flag and df['PriceNanFlag'][i] == False:
            flag=False
            missing_count_dict[count] += 1
            count=0
    print(missing_count_dict)   

In [33]:
missing_freq_count()

{0: 0, 1: 18305, 2: 3178, 3: 1162, 4: 507, 5: 269, 6: 187, 7: 92, 8: 70, 9: 53, 10: 52, 11: 38, 12: 30, 13: 33, 14: 11, 15: 22, 16: 13, 17: 16, 18: 7, 19: 11, 20: 11, 21: 6, 22: 5, 23: 5, 24: 7, 25: 6, 26: 4, 27: 4, 28: 5, 29: 6, 30: 10, 31: 8, 32: 3, 33: 6, 34: 5, 35: 4, 36: 3, 37: 3, 38: 6, 39: 5, 40: 3, 41: 1, 42: 2, 43: 3, 44: 0, 45: 2, 46: 0, 47: 2, 48: 1, 49: 2, 50: 1, 51: 0, 52: 1, 53: 1, 54: 0, 55: 1, 56: 3, 57: 0, 58: 1, 59: 0, 60: 1, 61: 0, 62: 1, 63: 0, 64: 2, 65: 2, 66: 1, 67: 0, 68: 1, 69: 0, 70: 0, 71: 0, 72: 1, 73: 0, 74: 0, 75: 0, 76: 0, 77: 1, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 1, 85: 0, 86: 0, 87: 0, 88: 1, 89: 0, 90: 0, 91: 1, 92: 0, 93: 0, 94: 1, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 1, 101: 1, 102: 0, 103: 0, 104: 1, 105: 0, 106: 1, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 1, 117: 0, 118: 0, 119: 0, 120: 1, 121: 1, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0,

In [35]:
172731/43

4017.0

In [54]:
def fill_single_miss():
    i=1
    count=0
    previous_missing=False
    next_missing = False
    current_missing= False
    while(i<df.shape[0]-1):
        current_missing=df['PriceNanFlag'][i]
        previous_missing = df['PriceNanFlag'][i-1]
        next_missing = df['PriceNanFlag'][i+1]
        
#         print('d')
        if current_missing==True and previous_missing==False and next_missing==False:
            df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+1])/2
            df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+1])/2
#             print('h')
            count+=1
        i+=1
    return count
        

In [55]:
fill_single_miss()

/tmp/ipykernel_2475335/3791241559.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+1])/2
/tmp/ipykernel_2475335/3791241559.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+1])/2


18305

In [64]:
def fill_two_misses():
    i=1
    count=0
    previous_missing=False
    next_missing = False
    current_missing= False
    while(i<df.shape[0]-2):
        current_missing=df['PriceNanFlag'][i] and df['PriceNanFlag'][i+1]
        previous_missing = df['PriceNanFlag'][i-1]
        next_missing = df['PriceNanFlag'][i+2]
        
#         print('d')
        if current_missing==True and previous_missing==False and next_missing==False:
            df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+2])/2
            df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+2])/2
            df['Imp_Price'][i+1] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+2])/2
            df['Imp_Arrival'][i+1] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+2])/2
#             print('h')
            count+=1
        i+=1
    return count

In [65]:
fill_two_misses()

/tmp/ipykernel_2475335/3247656120.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+2])/2
/tmp/ipykernel_2475335/3247656120.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+2])/2
/tmp/ipykernel_2475335/3247656120.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+1] = (df['Imp_Price'

3178

In [66]:
def fill_three_misses():
    i=1
    count=0
    previous_missing=False
    next_missing = False
    current_missing= False
    while(i<df.shape[0]-3):
        current_missing=df['PriceNanFlag'][i] and df['PriceNanFlag'][i+1] and df['PriceNanFlag'][i+2]
        previous_missing = df['PriceNanFlag'][i-1]
        next_missing = df['PriceNanFlag'][i+3]
        
#         print('d')
        if current_missing==True and previous_missing==False and next_missing==False:
            df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+3])/2
            df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+3])/2
            df['Imp_Price'][i+1] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+3])/2
            df['Imp_Arrival'][i+1] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+3])/2
            df['Imp_Price'][i+2] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+3])/2
            df['Imp_Arrival'][i+2] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+3])/2
            count+=1
        i+=1
    return count

In [67]:
fill_three_misses()

/tmp/ipykernel_2475335/2811492243.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+3])/2
/tmp/ipykernel_2475335/2811492243.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+3])/2
/tmp/ipykernel_2475335/2811492243.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+1] = (df['Imp_Price'

1162

In [68]:
df[2050:2060]

,Market,Arrivals,Date,Modal_Price,District,Imp_Price,Imp_Arrival,PriceNanFlag,ArrivalNanFlag,latitude,...,Soil_Type,Humidity,Sunlight,Temperature,Avg_Precipitation,Last_Precipitation,Year,Month_name,Day,Month
2050,Agra,120.0,13-Aug-2013,2500.0,Agra,2500.0,120.0,False,False,27.25,...,1.999985,84.939757,1.353459e+07,301.766362,147888.435453,7.111579e+04,2013,Aug,13,8
2051,Agra,100.0,14-Aug-2013,3150.0,Agra,3150.0,100.0,False,False,27.25,...,1.999985,86.348095,1.117862e+07,301.481810,694674.906233,2.749004e+05,2013,Aug,14,8
2052,Agra,62.0,15-Aug-2013,3700.0,Agra,3700.0,62.0,False,False,27.25,...,1.999985,87.366731,9.127843e+06,301.331388,455157.133383,6.262586e+04,2013,Aug,15,8
2053,Agra,NaN,16-Aug-2013,NaN,Agra,4175.0,48.5,True,True,27.25,...,1.999985,90.381171,8.402299e+06,300.942450,709490.067333,8.002818e+05,2013,Aug,16,8
2054,Agra,NaN,17-Aug-2013,NaN,Agra,4175.0,48.5,True,True,27.25,...,1.999985,88.406566,7.693792e+06,300.626735,349240.844711,1.793764e+05,2013,Aug,17,8
2055,Agra,NaN,18-Aug-2013,NaN,Agra,4175.0,48.5,True,True,27.25,...,1.999985,85.559958,9.264190e+06,301.239420,471830.054679,1.305498e+06,2013,Aug,18,8
2056,Agra,35.0,19-Aug-2013,4650.0,Agra,4650.0,35.0,False,False,27.25,...,1.999985,87.833337,7.190536e+06,301.487153,510569.975825,7.408448e+05,2013,Aug,19,8
2057,Agra,32.0,20-Aug-2013,4680.0,Agra,4680.0,32.0,False,False,27.25,...,1.999985,83.959571,1.237486e+07,302.820906,270389.020245,2.016872e+04,2013,Aug,20,8
2058,Agra,36.0,21-Aug-2013,4400.0,Agra,4400.0,36.0,False,False,27.25,...,1.999985,87.788132,1.253387e+07,302.236512,900095.648542,5.699620e+05,2013,Aug,21,8
2059,Agra,46.0,22-Aug-2013,4200.0,Agra,4200.0,46.0,False,False,27.25,...,1.999985,85.167239,9.824776e+06,302.358153,605121.798046,3.725290e+00,2013,Aug,22,8


In [70]:
    i=1
    count=0
    previous_missing=False
    next_missing = False
    current_missing= False
    while(i<df.shape[0]-4):
        if count<20:
            current_missing=df['PriceNanFlag'][i] and df['PriceNanFlag'][i+1] and df['PriceNanFlag'][i+2] and df['PriceNanFlag'][i+3]
            previous_missing = df['PriceNanFlag'][i-1]
            next_missing = df['PriceNanFlag'][i+4]

    #         print('d')
            if current_missing==True and previous_missing==False and next_missing==False:
                print('pre: ', df['Imp_Price'][i-1],' next: ',df['Imp_Price'][i+4],'  Imputes: ',df['Imp_Price'][i],df['Imp_Price'][i+1],df['Imp_Price'][i+2],df['Imp_Price'][i+3])
                count +=1
        i+=1

pre:  820.0  next:  850.0   Imputes:  868.8706211875133 841.9062320224659 877.0948175256357 813.9437160964762
pre:  550.0  next:  725.0   Imputes:  672.7653013297572 647.855137858395 662.4195412498348 658.2844193966116
pre:  975.0  next:  965.0   Imputes:  943.534498239319 936.6814562323916 922.9374324581528 940.4265079404252
pre:  975.0  next:  1050.0   Imputes:  958.9072503976012 912.3912549726174 915.2069892337922 951.4721661943552
pre:  3800.0  next:  3780.0   Imputes:  2865.872022830417 3567.232133013794 3212.789340192861 3931.209146664847
pre:  1200.0  next:  1170.0   Imputes:  1157.2707516526689 1099.435647593655 1142.9757269787765 1065.1827888721884
pre:  1670.0  next:  1680.0   Imputes:  1626.5293217023643 1461.9362727452617 1588.7359368519856 1523.142733328549
pre:  1020.0  next:  1020.0   Imputes:  978.7435892608758 988.628351966338 880.9452814097494 961.4927940472936
pre:  980.0  next:  980.0   Imputes:  954.7826908690512 939.4920471600268 958.906077299308 918.7249788921036

In [78]:
def fill_misses(n,check=False):
    i=1
    count=0
    previous_missing=False
    next_missing = False
    current_missing= False
    while(i<df.shape[0]-n):
        flag=True
        for j in range(n):
            flag= flag and df['PriceNanFlag'][i+j]
        current_missing = flag
        previous_missing = df['PriceNanFlag'][i-1]
        next_missing = df['PriceNanFlag'][i+n]
        
#         print('d')
        if current_missing==True and previous_missing==False and next_missing==False:
            for j in range(n):
                if check:
                   
                    if (df['Imp_Price'][i+j]<df['Imp_Price'][i-1] and df['Imp_Price'][i+j]>df['Imp_Price'][i+n]) or (df['Imp_Price'][i+j]>df['Imp_Price'][i-1] and df['Imp_Price'][i+j]<df['Imp_Price'][i+n]) == False:
                        df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
                        df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2
                else:    
                    df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
                    df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2

            count+=1
        i+=1
    return count

In [76]:
fill_misses(3)

/tmp/ipykernel_2475335/2252332365.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
/tmp/ipykernel_2475335/2252332365.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2


1162

In [79]:
fill_misses(4)

/tmp/ipykernel_2475335/183770110.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
/tmp/ipykernel_2475335/183770110.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2


507

In [80]:
fill_misses(5,True)

/tmp/ipykernel_2475335/183770110.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
/tmp/ipykernel_2475335/183770110.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2


269

In [81]:
fill_misses(6,True)

/tmp/ipykernel_2475335/183770110.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Price'][i+j] = (df['Imp_Price'][i-1]+df['Imp_Price'][i+n])/2
/tmp/ipykernel_2475335/183770110.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Imp_Arrival'][i+j] = (df['Imp_Arrival'][i-1]+df['Imp_Arrival'][i+n])/2


186

In [82]:
df.head()

,Market,Arrivals,Date,Modal_Price,District,Imp_Price,Imp_Arrival,PriceNanFlag,ArrivalNanFlag,latitude,...,Soil_Type,Humidity,Sunlight,Temperature,Avg_Precipitation,Last_Precipitation,Year,Month_name,Day,Month
0,Agra,NaN,01-Jan-2008,NaN,Agra,533.577743,136.448320,True,True,27.25,...,1.999985,53.146736,1.265486e+07,289.903276,3.72529,3.72529,2008,Jan,1,1
1,Agra,NaN,02-Jan-2008,NaN,Agra,564.358912,141.135159,True,True,27.25,...,1.999985,55.914157,1.211169e+07,289.533809,3.72529,3.72529,2008,Jan,2,1
2,Agra,NaN,03-Jan-2008,NaN,Agra,562.825324,145.897888,True,True,27.25,...,1.999985,61.595423,1.274031e+07,288.712016,3.72529,3.72529,2008,Jan,3,1
3,Agra,NaN,04-Jan-2008,NaN,Agra,560.865227,220.000000,True,True,27.25,...,1.999985,59.810162,1.259190e+07,289.971033,3.72529,3.72529,2008,Jan,4,1
4,Agra,NaN,05-Jan-2008,NaN,Agra,538.705688,218.330750,True,True,27.25,...,1.999985,55.464904,1.240704e+07,291.377191,3.72529,3.72529,2008,Jan,5,1


## Adding new features

In [84]:
def add_last_3_days_price_and_arrival():
    last_day_price= np.zeros((df.shape[0]))
    second_last_day_price= np.zeros((df.shape[0]))
    third_last_day_price = np.zeros((df.shape[0]))
    last_week_avg_price = np.zeros((df.shape[0]))

    last_day_arrival= np.zeros((df.shape[0]))
    second_last_day_arrival= np.zeros((df.shape[0]))
    third_last_day_arrival = np.zeros((df.shape[0]))
    last_week_avg_arrival = np.zeros((df.shape[0]))

    i=4017

    for j in range(0,df.shape[0],i):
        last_day_price[j] = df['Imp_Price'][j]
        last_day_arrival[j]=  df['Imp_Arrival'][j]

        second_last_day_price[j] = df['Imp_Price'][j]
        second_last_day_arrival[j]=  df['Imp_Arrival'][j]
        second_last_day_price[j+1] = df['Imp_Price'][j]
        second_last_day_arrival[j+1]=  df['Imp_Arrival'][j]

        third_last_day_price[j] = df['Imp_Price'][j]
        third_last_day_arrival[j]=  df['Imp_Arrival'][j]
        third_last_day_price[j+1] = df['Imp_Price'][j]
        third_last_day_arrival[j+1]=  df['Imp_Arrival'][j]
        third_last_day_price[j+2] = df['Imp_Price'][j+1]
        third_last_day_arrival[j+2]=  df['Imp_Arrival'][j+1]

        for k in range(j,j+7):
            last_week_avg_price[k] = df['Imp_Price'][j]
            last_week_avg_arrival[k]=  df['Imp_Arrival'][j]


    cur_price_sum =0
    cur_arrival_sum=0


    for i in range(df.shape[0]):

        cur_price_sum += df['Imp_Price'][i]
        cur_arrival_sum += df['Imp_Arrival'][i]

    #     print(i,'  : ',cur_price_sum)
        if i%4017 != 0:
            last_day_price[i] = df['Imp_Price'][i-1]
            last_day_arrival[i]=  df['Imp_Arrival'][i-1]

            if i%4017 !=1:
                second_last_day_price[i] = df['Imp_Price'][i-2]
                second_last_day_arrival[i]=  df['Imp_Arrival'][i-2]

                if i%4017 !=2:
                    third_last_day_price[i] = df['Imp_Price'][i-3]
                    third_last_day_arrival[i]=  df['Imp_Arrival'][i-3]

                    if i%4017 !=3 and i%4017 !=4 and i%4017 !=5 and i%4017 !=6:
    #                     print(i,'  :***** ',cur_price_sum)
                        last_week_avg_price[i] = (cur_price_sum/7)
                        last_week_avg_arrival[i]=  (cur_arrival_sum/7)
                        cur_price_sum -= df['Imp_Price'][i-7]
                        cur_arrival_sum -= df['Imp_Arrival'][i-7]
                    
    df['last_day_price'] = last_day_price
    df['second_last_day_price'] = second_last_day_price
    df['third_last_day_price'] = third_last_day_price
    df['last_week_avg_price'] = last_week_avg_price
    df['last_day_arrival'] = last_day_arrival
    df['second_last_day_arrival'] = second_last_day_arrival
    df['third_last_day_arrival'] = third_last_day_arrival
    df['last_week_avg_arrival'] = last_week_avg_arrival

In [86]:
add_last_3_days_price_and_arrival()

In [92]:
def add_last_3_days_precipitation():
    last_day_Precipitation= np.zeros((df.shape[0]))
    second_last_day_Precipitation= np.zeros((df.shape[0]))
    third_last_day_Precipitation = np.zeros((df.shape[0]))
    last_week_avg_Precipitation = np.zeros((df.shape[0])) 

    i=4017
    for j in range(0,df.shape[0]-7,i):
        last_day_Precipitation[j] = df['Avg_Precipitation'][j]

        second_last_day_Precipitation[j] = df['Avg_Precipitation'][j]
        second_last_day_Precipitation[j+1] = df['Avg_Precipitation'][j]

        third_last_day_Precipitation[j] = df['Avg_Precipitation'][j]
        third_last_day_Precipitation[j+1] = df['Avg_Precipitation'][j]
        third_last_day_Precipitation[j+2] = df['Avg_Precipitation'][j+1]

        for k in range(j,j+7):
            last_week_avg_Precipitation[k] = df['Avg_Precipitation'][j]

    cur_Precipitation_sum =0


    for i in range(df.shape[0]):

        cur_Precipitation_sum += df['Avg_Precipitation'][i]


        if i%4017 != 0:
            last_day_Precipitation[i] = df['Avg_Precipitation'][i-1]

            if i%4017 !=1:
                second_last_day_Precipitation[i] = df['Avg_Precipitation'][i-2]

                if i%4017 !=2:
                    third_last_day_Precipitation[i] = df['Avg_Precipitation'][i-3]

                    if i%4017 !=3 and i%4017 !=4 and i%4017 !=5 and i%4017 !=6:

                        last_week_avg_Precipitation[i] = (cur_Precipitation_sum/7)
                        cur_Precipitation_sum -= df['Avg_Precipitation'][i-7]



    df['last_day_Precipitation'] = last_day_Precipitation
    df['second_last_day_Precipitation'] = second_last_day_Precipitation
    df['third_last_day_Precipitation'] = third_last_day_Precipitation
    df['last_week_avg_Precipitation'] = last_week_avg_Precipitation

In [93]:
add_last_3_days_precipitation()

In [94]:
df.head()

,Market,Arrivals,Date,Modal_Price,District,Imp_Price,Imp_Arrival,PriceNanFlag,ArrivalNanFlag,latitude,...,third_last_day_price,last_week_avg_price,last_day_arrival,second_last_day_arrival,third_last_day_arrival,last_week_avg_arrival,last_day_Precipitation,second_last_day_Precipitation,third_last_day_Precipitation,last_week_avg_Precipitation
0,Agra,NaN,01-Jan-2008,NaN,Agra,533.577743,136.448320,True,True,27.25,...,533.577743,533.577743,136.448320,136.448320,136.448320,136.44832,3.72529,3.72529,3.72529,3.72529
1,Agra,NaN,02-Jan-2008,NaN,Agra,564.358912,141.135159,True,True,27.25,...,533.577743,533.577743,136.448320,136.448320,136.448320,136.44832,3.72529,3.72529,3.72529,3.72529
2,Agra,NaN,03-Jan-2008,NaN,Agra,562.825324,145.897888,True,True,27.25,...,564.358912,533.577743,141.135159,136.448320,141.135159,136.44832,3.72529,3.72529,3.72529,3.72529
3,Agra,NaN,04-Jan-2008,NaN,Agra,560.865227,220.000000,True,True,27.25,...,533.577743,533.577743,145.897888,141.135159,136.448320,136.44832,3.72529,3.72529,3.72529,3.72529
4,Agra,NaN,05-Jan-2008,NaN,Agra,538.705688,218.330750,True,True,27.25,...,564.358912,533.577743,220.000000,145.897888,141.135159,136.44832,3.72529,3.72529,3.72529,3.72529


In [95]:
df.to_csv('updated_impute_features_added_post_impute_environmental_43_markets_2008_2018.csv')

## Make input

In [96]:
def make_market_array():
    market_names = df['Market'].unique()
    market_array = np.zeros([df.shape[0],len(market_names)])
    market_names  = list(market_names)
    for i in range(df.shape[0]):
        market = df['Market'][i]
        index = market_names.index(market)
        market_array[i][index] =1
    return market_array

In [98]:
def make_day_array():
    day_array = np.zeros([df.shape[0],31])

    for i in range(df.shape[0]):
        day = df['Day'][i]
        day_array[i][day-1] =1

    return day_array

In [99]:
def make_month_array():
    month_array =  np.zeros([df.shape[0],12])

    for i in range(df.shape[0]):
        month = df['Month'][i]
        month = int(month)
        month_array[i][month-1] =1

    return month_array

In [100]:
def make_price_nan_arrival_nan_array():
    PriceNanFlag_array = np.array(df['PriceNanFlag'])


    for i in range(len(PriceNanFlag_array)):
        if PriceNanFlag_array[i] ==True:
            PriceNanFlag_array[i]=1
        else:
            PriceNanFlag_array[i]=0

    PriceNanFlag_array = list(PriceNanFlag_array)

    PriceNanFlag_array = [0]+PriceNanFlag_array[:len(PriceNanFlag_array)-1]
    
    ArrivalNanFlag_array = np.array(df['ArrivalNanFlag'])


    for i in range(len(ArrivalNanFlag_array)):
        if ArrivalNanFlag_array[i] ==True:
            ArrivalNanFlag_array[i]=1
        else:
            ArrivalNanFlag_array[i]=0

    ArrivalNanFlag_array = list(ArrivalNanFlag_array)

    ArrivalNanFlag_array = [0]+ArrivalNanFlag_array[:len(ArrivalNanFlag_array)-1]
    
    PriceNanFlag_array = np.array(PriceNanFlag_array)
    ArrivalNanFlag_array = np.array(ArrivalNanFlag_array)


    PriceNanFlag_array = PriceNanFlag_array.reshape([len(PriceNanFlag_array),1])
    ArrivalNanFlag_array = ArrivalNanFlag_array.reshape([len(ArrivalNanFlag_array),1])
    
    return PriceNanFlag_array,ArrivalNanFlag_array

In [104]:
def make_final_input():
    market_array = make_market_array()
    day_array = make_day_array()
    month_array = make_month_array()
    PriceNanFlag_array,ArrivalNanFlag_array = make_price_nan_arrival_nan_array()
    
    
    input_ = df.values[:,[9,10,11,12,13,14,15,16, 21,22,23,24,25,26,27,28,29,30,31,32]]
    Final_input = np.concatenate((input_,PriceNanFlag_array,ArrivalNanFlag_array,market_array,month_array,day_array),axis=1)
    
    
    return Final_input

In [105]:
Final_input = make_final_input()

In [101]:
Y= df.values[:,5]

In [106]:
Final_input.shape

(172731, 108)

In [279]:
def min_max_scale(Final_input=Final_input):
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(Final_input)
    
    return X,min_max_scaler

In [109]:
from sklearn import preprocessing
X = min_max_scale()

In [112]:
def time_based_split():
    X_train =[]
    X_test=[]
    Y_train=[]
    Y_test=[]
    years_df = list(df['Year'])
    
    for i in range(len(years_df)):
        if years_df[i] ==2018 or years_df[i]==2017:
            X_test.append(X[i])
            Y_test.append(Y[i])
        else:
            X_train.append(X[i])
            Y_train.append(Y[i])
            
    return X_train,Y_train,X_test,Y_test

In [113]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [114]:
len(X_train),len(X_test)

(141341, 31390)

In [120]:
from sklearn import linear_model
def fit_lasso(X_train,Y_train,X_test,Y_test):
 
    rgsrL = linear_model.Lasso(alpha=0.1)
    rgsrL.fit(X_train,Y_train)

    y_pred= rgsrL.predict(X_test)

    y_pred_train = rgsrL.predict(X_train)
    print('*'*100)
    print("TRAIN ERROR")

    print('MAE:', metrics.mean_absolute_error(Y_train, y_pred_train))  
    print('MSE:', metrics.mean_squared_error(Y_train, y_pred_train))  
    print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_train, y_pred_train)))
    
    print('*'*100)
    print("TEST ERROR")
    
    print('MAE:', metrics.mean_absolute_error(Y_test, y_pred))  
    print('MSE:', metrics.mean_squared_error(Y_test, y_pred))  
    print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
    
    print('*'*100)
    for i in range(10):
        print('Pred: ',y_pred[i],' True: ',Y_test[i])

In [234]:
def fit_rf(X_train,Y_train,X_test,Y_test, n_estimators=100,max_depth=7,verbose=2):
 
    regressor = RandomForestRegressor(n_estimators = 100,max_depth=7,verbose=2, random_state = 0)

    # fit the regressor with x and y data
    regressor.fit(X_train, Y_train)

    y_pred = regressor.predict(X_test) 
    y_pred_train = regressor.predict(X_train)
    
    print('*'*100)
    print("TRAIN ERROR")

    print('MAE:', metrics.mean_absolute_error(Y_train, y_pred_train))  
    print('MSE:', metrics.mean_squared_error(Y_train, y_pred_train))  
    print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_train, y_pred_train)))
    
    print('*'*100)
    print("TEST ERROR")
    
    print('MAE:', metrics.mean_absolute_error(Y_test, y_pred))  
    print('MSE:', metrics.mean_squared_error(Y_test, y_pred))  
    print('RMSE:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
    
    print('*'*100)
    for i in range(10):
        print('Pred: ',y_pred[i],' True: ',Y_test[i])
        
    return regressor

In [123]:
fit_lasso(X_train,Y_train,X_test,Y_test)

****************************************************************************************************
TRAIN ERROR
MAE: 33.75592065385851
MSE: 11507.450763415764
RMSE: 107.27278668616643
****************************************************************************************************
TEST ERROR
MAE: 39.118088510773426
MSE: 17199.744815535425
RMSE: 131.14779760078102
****************************************************************************************************
Pred:  867.2005070616894  True:  855.0
Pred:  858.6096278111019  True:  850.0
Pred:  852.9046456265659  True:  840.0
Pred:  843.7624428044023  True:  830.0
Pred:  833.5255853503946  True:  810.0
Pred:  814.7646901061053  True:  800.0
Pred:  805.2646323086196  True:  810.0
Pred:  810.3774993936565  True:  825.0
Pred:  821.6650577929884  True:  840.0
Pred:  833.650897650823  True:  820.0


In [124]:
fit_rf(X_train,Y_train,X_test,Y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 31.12395348166862
MSE: 8127.815488357081
RMSE: 90.15439805332339
****************************************************************************************************
TEST ERROR
MAE: 40.11714491950967
MSE: 14689.348342468074
RMSE: 121.19962187427845
****************************************************************************************************
Pred:  862.875085294231  True:  855.0
Pred:  857.6706120372768  True:  850.0
Pred:  857.4613789407422  True:  840.0
Pred:  852.8728159716958  True:  830.0
Pred:  819.7947809430765  True:  810.0
Pred:  811.1709044007289  True:  800.0
Pred:  807.0929273175633  True:  810.0
Pred:  811.1709044007289  True:  825.0
Pred:  818.1751629291379  True:  840.0
Pred:  892.0966187356426  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


In [125]:
fit_rf(X_train,Y_train,X_test,Y_test,150,9,2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 31.12395348166862
MSE: 8127.815488357081
RMSE: 90.15439805332339
****************************************************************************************************
TEST ERROR
MAE: 40.11714491950967
MSE: 14689.348342468074
RMSE: 121.19962187427845
****************************************************************************************************
Pred:  862.875085294231  True:  855.0
Pred:  857.6706120372768  True:  850.0
Pred:  857.4613789407422  True:  840.0
Pred:  852.8728159716958  True:  830.0
Pred:  819.7947809430765  True:  810.0
Pred:  811.1709044007289  True:  800.0
Pred:  807.0929273175633  True:  810.0
Pred:  811.1709044007289  True:  825.0
Pred:  818.1751629291379  True:  840.0
Pred:  892.0966187356426  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


In [126]:
prices = list(df['Imp_Price'])

In [128]:
print('MAE:', metrics.mean_absolute_error(prices[1:], prices[:len(prices)-1]))  
print('MSE:', metrics.mean_squared_error(prices[1:], prices[:len(prices)-1]))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(prices[1:], prices[:len(prices)-1])))

MAE: 30.198885384946667
MSE: 13350.408650849478
RMSE: 115.54396847455725


In [129]:
def remove_outliers(X_train,Y_train,X_test,Y_test,up,low):
    X_train_95 = X_train.copy()
    X_test_95 = X_test.copy()
    Y_train_95 = Y_train.copy()
    Y_test_95 = Y_test.copy()

    high =np.percentile(Y, up)
    low = np.percentile(Y,low)

    outlier_indexes=[]


    for i in range(len(X_train)):
        if Y_train[i]<=low or Y_train[i]>=high:
            outlier_indexes.append(i)


    for index in sorted(outlier_indexes, reverse=True):
        del X_train_95[index]
        del Y_train_95[index]


    outlier_indexes=[]

    for i in range(len(X_test)):
        if Y_test[i]<=low or Y_test[i]>=high:
            outlier_indexes.append(i)


    for index in sorted(outlier_indexes, reverse=True):
        del X_test_95[index]
        del Y_test_95[index]
        
    return X_train_95,X_test_95,Y_train_95,Y_test_95

In [130]:
X_train_95,X_test_95,Y_train_95,Y_test_95 = remove_outliers(X_train,Y_train,X_test,Y_test,95,5)

In [132]:
fit_lasso(X_train_95,Y_train_95,X_test_95,Y_test_95)

****************************************************************************************************
TRAIN ERROR
MAE: 30.880395148898025
MSE: 6131.557512422532
RMSE: 78.30426241541728
****************************************************************************************************
TEST ERROR
MAE: 39.09156898554603
MSE: 10538.51688499837
RMSE: 102.6572787726149
****************************************************************************************************
Pred:  879.6664116925865  True:  855.0
Pred:  868.541509984597  True:  850.0
Pred:  862.4113161706259  True:  840.0
Pred:  853.5593299471107  True:  830.0
Pred:  843.4559609996638  True:  810.0
Pred:  824.9842567982379  True:  800.0
Pred:  816.9764692348751  True:  810.0
Pred:  820.1758824909556  True:  825.0
Pred:  828.4733469863011  True:  840.0
Pred:  840.9086461648193  True:  820.0


In [133]:
fit_rf(X_train_95,Y_train_95,X_test_95,Y_test_95)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 24.16884600962377
MSE: 3327.4062099630205
RMSE: 57.68367368643419
****************************************************************************************************
TEST ERROR
MAE: 35.26514467017232
MSE: 5829.832703250459
RMSE: 76.35334114006052
****************************************************************************************************
Pred:  845.5307576050405  True:  855.0
Pred:  842.5238409338193  True:  850.0
Pred:  841.9743919223097  True:  840.0
Pred:  840.1171270840596  True:  830.0
Pred:  837.7978824795083  True:  810.0
Pred:  826.6857021487123  True:  800.0
Pred:  805.7708543670825  True:  810.0
Pred:  809.1416971589498  True:  825.0
Pred:  829.7024190519384  True:  840.0
Pred:  840.0205286088051  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [135]:
Y


array([533.577743091078, 564.358912379708, 562.8253237470766, ..., 1150.0,
       1125.0, 615.056990106931], dtype=object)

In [137]:
with open('Y_price_updated_impute_95.npy', 'wb') as f:
    np.save(f, Y)

In [176]:
len(X_train)

141341

In [177]:
districts_array = df['District']

In [178]:
districts_array.shape

(172731,)

In [180]:
districts_array = np.array(districts_array)

In [182]:
districts_array

array(['Agra', 'Agra', 'Agra', ..., 'Unnao', 'Unnao', 'Unnao'],
      dtype=object)

# Embedded input

In [183]:
df_market_embedding = pd.read_csv('market_embedding.csv')
df_market_embedding = df_market_embedding.drop(['Unnamed: 0'],axis=1)
df_market_embedding = df_market_embedding.reset_index(drop=True)

markets = df['Market'].unique()
markets = list(markets)

market_array= df['Market']

market_input=[]
for i in range(df.shape[0]):
    market = df['Market'][i]
    index = markets.index(market)
    market_input.append([df_market_embedding['C1'][index],df_market_embedding['C2'][index],df_market_embedding['C3'][index]])
    

In [207]:
df_month_embedding = pd.read_csv('month_embedding.csv')
df_month_embedding = df_month_embedding.drop(['Unnamed: 0'],axis=1)
df_month_embedding = df_month_embedding.reset_index(drop=True)

months = df['Month'].unique()
months = list(months)

month_array= df['Month']


month_input=[]
for i in range(df.shape[0]):
    month = df['Month'][i]
    index = months.index(month)
    month_input.append([df_month_embedding['C1'][index],df_month_embedding['C2'][index],df_month_embedding['C3'][index]])
    

In [215]:
df_day_embedding = pd.read_csv('day_embedding.csv')
df_day_embedding = df_day_embedding.drop(['Unnamed: 0'],axis=1)
df_day_embedding = df_day_embedding.reset_index(drop=True)

days = df['Day'].unique()
days = list(days)

day_array= df['Day']


day_input=[]
for i in range(df.shape[0]):
    day = df['Day'][i]
    index = days.index(day)
    day_input.append([df_day_embedding['C1'][index],df_day_embedding['C2'][index],df_day_embedding['C3'][index]])
    

In [220]:
def make_final_embedded_input():
    market_array = np.array(market_input)
    day_array =np.array(day_input)
    month_array = np.array(month_input)
    PriceNanFlag_array,ArrivalNanFlag_array = make_price_nan_arrival_nan_array()
    
    
    input_ = df.values[:,[9,10,11,12,13,14,15,16, 21,22,23,24,25,26,27,28,29,30,31,32]]
    Final_input = np.concatenate((input_,PriceNanFlag_array,ArrivalNanFlag_array,market_array,month_array,day_array),axis=1)
    
    
    return Final_input

In [223]:
Final_input = make_final_embedded_input()

In [224]:
Final_input.shape

(172731, 31)

In [225]:
Y= df.values[:,5]

In [226]:
X = min_max_scale()

In [227]:
X.shape

(172731, 31)

In [228]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [229]:
fit_lasso(X_train,Y_train,X_test,Y_test)

****************************************************************************************************
TRAIN ERROR
MAE: 33.443880343992134
MSE: 11545.005282022725
RMSE: 107.44768625718622
****************************************************************************************************
TEST ERROR
MAE: 39.006224901371425
MSE: 17235.59101728453
RMSE: 131.28438984618288
****************************************************************************************************
Pred:  871.664056100824  True:  855.0
Pred:  864.1841375148522  True:  850.0
Pred:  858.2127269195737  True:  840.0
Pred:  848.8738027410468  True:  830.0
Pred:  838.6209646847656  True:  810.0
Pred:  817.4495810569673  True:  800.0
Pred:  810.1228261967483  True:  810.0
Pred:  814.7745267935014  True:  825.0
Pred:  823.0627240991472  True:  840.0
Pred:  833.3006014185547  True:  820.0


In [230]:
fit_rf(X_train,Y_train,X_test,Y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 31.040030517370866
MSE: 8129.444006187384
RMSE: 90.16342942783056
****************************************************************************************************
TEST ERROR
MAE: 39.07817429109518
MSE: 14312.802780141543
RMSE: 119.6361265677786
****************************************************************************************************
Pred:  870.4017561274162  True:  855.0
Pred:  864.3625500960594  True:  850.0
Pred:  864.1533169995248  True:  840.0
Pred:  859.7520501660745  True:  830.0
Pred:  819.9954622032025  True:  810.0
Pred:  811.2456471138353  True:  800.0
Pred:  807.1676700306696  True:  810.0
Pred:  811.2456471138353  True:  825.0
Pred:  818.3758441892638  True:  840.0
Pred:  902.4958421737666  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


In [231]:
X_train_95,X_test_95,Y_train_95,Y_test_95 = remove_outliers(X_train,Y_train,X_test,Y_test,95,5)

In [235]:
rgsrr = fit_rf(X_train_95,Y_train_95,X_test_95,Y_test_95)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 24.15015584094444
MSE: 3335.49098696107
RMSE: 57.75370972466678
****************************************************************************************************
TEST ERROR
MAE: 34.10698429669753
MSE: 5438.611603393961
RMSE: 73.74694301050017
****************************************************************************************************
Pred:  845.5307576050407  True:  855.0
Pred:  842.5238409338194  True:  850.0
Pred:  841.9743919223099  True:  840.0
Pred:  840.1171270840599  True:  830.0
Pred:  837.7978824795084  True:  810.0
Pred:  826.6857021487125  True:  800.0
Pred:  806.2092010214236  True:  810.0
Pred:  809.5800438132908  True:  825.0
Pred:  829.8190834358937  True:  840.0
Pred:  840.0205286088053  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


In [239]:
importances = rgsrr.feature_importances_
importances = np.multiply(importances,10000000)

for i in range(len(importances)):
    print('i: ',i,' importance: ',importances[i])

i:  0  importance:  239.72767307746253
i:  1  importance:  297.69352513862316
i:  2  importance:  5.0772335501245465
i:  3  importance:  798.2687352251495
i:  4  importance:  448.5159133991192
i:  5  importance:  869.1196052473144
i:  6  importance:  289.12016585151537
i:  7  importance:  288.9658792711182
i:  8  importance:  9783673.302974535
i:  9  importance:  185426.731437357
i:  10  importance:  20688.652540610852
i:  11  importance:  894.7416736189479
i:  12  importance:  944.7065591343342
i:  13  importance:  495.6101608901116
i:  14  importance:  447.05118368948064
i:  15  importance:  360.7239654357006
i:  16  importance:  359.34039631439714
i:  17  importance:  192.71831484368946
i:  18  importance:  122.86189932815728
i:  19  importance:  663.9528739488256
i:  20  importance:  12.592339450055178
i:  21  importance:  9.351172348018931
i:  22  importance:  224.99184233068664
i:  23  importance:  198.561934329706
i:  24  importance:  241.67699888976864
i:  25  importance:  397.

In [241]:
rgsrr.predict(X_test_95[0].reshape(1, -1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([845.53075761])

In [244]:
indexes=[9,10,11,12,13,14,15,16, 21,22,23,24,25,26,27,28,29,30,31,32]
features = list(df.columns)
input_features =[]

for i in indexes:
    input_features.append(features[i])

In [245]:
input_features

['latitude',
 'longitude',
 'Soil_Type',
 'Humidity',
 'Sunlight',
 'Temperature',
 'Avg_Precipitation',
 'Last_Precipitation',
 'last_day_price',
 'second_last_day_price',
 'third_last_day_price',
 'last_week_avg_price',
 'last_day_arrival',
 'second_last_day_arrival',
 'third_last_day_arrival',
 'last_week_avg_arrival',
 'last_day_Precipitation',
 'second_last_day_Precipitation',
 'third_last_day_Precipitation',
 'last_week_avg_Precipitation']

In [249]:
for i in range(len(input_features)):
    print(indexes[i],' : ',input_features[i],' : ', importances[i])

9  :  latitude  :  239.72767307746253
10  :  longitude  :  297.69352513862316
11  :  Soil_Type  :  5.0772335501245465
12  :  Humidity  :  798.2687352251495
13  :  Sunlight  :  448.5159133991192
14  :  Temperature  :  869.1196052473144
15  :  Avg_Precipitation  :  289.12016585151537
16  :  Last_Precipitation  :  288.9658792711182
21  :  last_day_price  :  9783673.302974535
22  :  second_last_day_price  :  185426.731437357
23  :  third_last_day_price  :  20688.652540610852
24  :  last_week_avg_price  :  894.7416736189479
25  :  last_day_arrival  :  944.7065591343342
26  :  second_last_day_arrival  :  495.6101608901116
27  :  third_last_day_arrival  :  447.05118368948064
28  :  last_week_avg_arrival  :  360.7239654357006
29  :  last_day_Precipitation  :  359.34039631439714
30  :  second_last_day_Precipitation  :  192.71831484368946
31  :  third_last_day_Precipitation  :  122.86189932815728
32  :  last_week_avg_Precipitation  :  663.9528739488256


In [250]:
def make_LSTMs_input():
    market_array = np.array(market_input)
    day_array =np.array(day_input)
    month_array = np.array(month_input)
    PriceNanFlag_array,ArrivalNanFlag_array = make_price_nan_arrival_nan_array()
    
    
    input_lstms = df.values[:,[9,10,12,14,15,21,24,25,28,29,32]]
    Final_input_lstms = np.concatenate((input_lstms,PriceNanFlag_array,ArrivalNanFlag_array,market_array,month_array,day_array),axis=1)
    
    
    return Final_input_lstms

In [251]:
Final_input_lstms = make_LSTMs_input()

In [252]:
Final_input_lstms.shape

(172731, 22)

In [253]:
with open('Final_input_lstms.npy', 'wb') as f:
    np.save(f, Final_input_lstms)

In [254]:
Y= df.values[:,5]

In [255]:
Y

array([533.577743091078, 564.358912379708, 562.8253237470766, ..., 1150.0,
       1125.0, 615.056990106931], dtype=object)

In [256]:
with open('Y_lstms.npy', 'wb') as f:
    np.save(f, Y)

In [257]:
X= Final_input_lstms

In [258]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [259]:
X_train[0]

array([27.25, 78.0, 53.14673604166666, 289.9032758333332,
       3.725290299999999, 533.577743091078, 533.577743091078,
       136.4483200418814, 136.4483200418814, 3.725290299999999,
       3.725290299999999, 0, 0, 0.75819767, -0.773561, -0.8356619,
       -0.014831018, -0.001299955, 0.019895706, 0.0024683475, 0.031677876,
       -0.02698052], dtype=object)

In [261]:
def save_npy(name,path):
    with open(path, 'wb') as f:
        np.save(f, name)

In [262]:
save_npy(X_train,'X_train_lstms.npy')
save_npy(X_test,'X_test_lstms.npy')
save_npy(Y_train,'Y_train_lstms.npy')
save_npy(Y_test,'Y_test_lstms.npy')

In [263]:
X_train_95,X_test_95,Y_train_95,Y_test_95 = remove_outliers(X_train,Y_train,X_test,Y_test,95,5)

In [264]:
save_npy(X_train_95,'X_train_95_lstms.npy')
save_npy(X_test_95,'X_test_95_lstms.npy')
save_npy(Y_train_95,'Y_train_95_lstms.npy')
save_npy(Y_test_95,'Y_test_95_lstms.npy')

In [265]:
df

,Market,Arrivals,Date,Modal_Price,District,Imp_Price,Imp_Arrival,PriceNanFlag,ArrivalNanFlag,latitude,...,third_last_day_price,last_week_avg_price,last_day_arrival,second_last_day_arrival,third_last_day_arrival,last_week_avg_arrival,last_day_Precipitation,second_last_day_Precipitation,third_last_day_Precipitation,last_week_avg_Precipitation
0,Agra,NaN,01-Jan-2008,NaN,Agra,533.577743,136.448320,True,True,27.25,...,533.577743,533.577743,136.448320,136.448320,136.448320,136.448320,3.72529,3.72529,3.72529,3.725290
1,Agra,NaN,02-Jan-2008,NaN,Agra,564.358912,141.135159,True,True,27.25,...,533.577743,533.577743,136.448320,136.448320,136.448320,136.448320,3.72529,3.72529,3.72529,3.725290
2,Agra,NaN,03-Jan-2008,NaN,Agra,562.825324,145.897888,True,True,27.25,...,564.358912,533.577743,141.135159,136.448320,141.135159,136.448320,3.72529,3.72529,3.72529,3.725290
3,Agra,NaN,04-Jan-2008,NaN,Agra,560.865227,220.000000,True,True,27.25,...,533.577743,533.577743,145.897888,141.135159,136.448320,136.448320,3.72529,3.72529,3.72529,3.725290
4,Agra,NaN,05-Jan-2008,NaN,Agra,538.705688,218.330750,True,True,27.25,...,564.358912,533.577743,220.000000,145.897888,141.135159,136.448320,3.72529,3.72529,3.72529,3.725290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172726,Unnao,15.8,27-Dec-2018,1150.0,Unnao,1150.000000,15.800000,False,False,26.50,...,1150.000000,39898.438561,17.400000,16.600000,15.800000,2301.037785,3.72529,3.72529,3.72529,160.719667
172727,Unnao,16.7,28-Dec-2018,1150.0,Unnao,1150.000000,16.700000,False,False,26.50,...,1150.000000,39891.295704,15.800000,17.400000,16.600000,2301.037785,3.72529,3.72529,3.72529,160.719667
172728,Unnao,21.8,29-Dec-2018,1150.0,Unnao,1150.000000,21.800000,False,False,26.50,...,1150.000000,39884.152847,16.700000,15.800000,17.400000,2302.094927,3.72529,3.72529,3.72529,160.719667
172729,Unnao,15.5,30-Dec-2018,1125.0,Unnao,1125.000000,15.500000,False,False,26.50,...,1150.000000,39873.438561,21.800000,16.700000,15.800000,2302.666356,3.72529,3.72529,3.72529,160.719667


In [266]:
rgsrr = fit_rf(X_train_95,Y_train_95,X_test_95,Y_test_95,150,9,2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   46.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 24.357667464687196
MSE: 3873.736243917499
RMSE: 62.23934642906767
****************************************************************************************************
TEST ERROR
MAE: 34.66745850381321
MSE: 5705.60268225298
RMSE: 75.53543990904521
****************************************************************************************************
Pred:  856.2551114081722  True:  855.0
Pred:  853.3753905157962  True:  850.0
Pred:  853.1657942865454  True:  840.0
Pred:  839.7070155352751  True:  830.0
Pred:  831.1304052752796  True:  810.0
Pred:  814.2720502976076  True:  800.0
Pred:  803.1787943784061  True:  810.0
Pred:  814.2720502976076  True:  825.0
Pred:  826.7889745140626  True:  840.0
Pred:  839.7070155352751  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [267]:
X_test_95[0]

array([27.25, 78.0, 79.19499237499998, 291.610795, 667.137411941667,
       860.0, 1073.5714285714287, 270.0, 268.2857142857143,
       3.725290299999999, 99.03063486240862, 0, 0, 0.75819767, -0.773561,
       -0.8356619, -0.014831018, -0.001299955, 0.019895706, 0.0024683475,
       0.031677876, -0.02698052], dtype=object)

In [268]:
X_train_95[0]

array([27.25, 78.0, 53.14673604166666, 289.9032758333332,
       3.725290299999999, 533.577743091078, 533.577743091078,
       136.4483200418814, 136.4483200418814, 3.725290299999999,
       3.725290299999999, 0, 0, 0.75819767, -0.773561, -0.8356619,
       -0.014831018, -0.001299955, 0.019895706, 0.0024683475, 0.031677876,
       -0.02698052], dtype=object)

In [280]:
X,min_max_scaler = min_max_scale(Final_input_lstms)

In [271]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [272]:
X_train_95,X_test_95,Y_train_95,Y_test_95 = remove_outliers(X_train,Y_train,X_test,Y_test,95,5)

In [273]:
rgsrr = fit_rf(X_train_95,Y_train_95,X_test_95,Y_test_95,150,9,2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   46.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 24.35743111144662
MSE: 3873.7486617441364
RMSE: 62.239446187640006
****************************************************************************************************
TEST ERROR
MAE: 34.667658505584946
MSE: 5705.940304965777
RMSE: 75.53767473893922
****************************************************************************************************
Pred:  856.2438769850468  True:  855.0
Pred:  853.4681674274414  True:  850.0
Pred:  853.1604608250458  True:  840.0
Pred:  839.7072174407829  True:  830.0
Pred:  831.1305193706953  True:  810.0
Pred:  814.2718585550474  True:  800.0
Pred:  803.1795311784674  True:  810.0
Pred:  814.2718585550474  True:  825.0
Pred:  826.7884177874832  True:  840.0
Pred:  839.7072174407829  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished


In [274]:
X_test[0]

array([4.50000000e-01, 7.69230769e-02, 7.77307827e-01, 2.77701203e-01,
       5.32127461e-05, 8.64321608e-02, 1.62540799e-02, 3.18121871e-03,
       5.39451632e-03, 0.00000000e+00, 2.92910795e-05, 0.00000000e+00,
       0.00000000e+00, 5.26684882e-01, 4.92561757e-01, 1.65344631e-01,
       6.19416003e-01, 5.47306968e-01, 5.25032511e-01, 0.00000000e+00,
       1.00000000e+00, 0.00000000e+00])

In [275]:
X = Final_input_lstms

In [276]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [277]:
X_test[0]

array([27.25, 78.0, 79.19499237499998, 291.610795, 667.137411941667,
       860.0, 1073.5714285714287, 270.0, 268.2857142857143,
       3.725290299999999, 99.03063486240862, 0, 0, 0.75819767, -0.773561,
       -0.8356619, -0.014831018, -0.001299955, 0.019895706, 0.0024683475,
       0.031677876, -0.02698052], dtype=object)

In [281]:
X = min_max_scaler.fit_transform(X_test[0].reshape(1,-1))

In [282]:
X

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

# Fitting without minmax

In [292]:
X =Final_input

In [372]:
X_train,Y_train,X_test,Y_test = time_based_split()

In [373]:
X_train_95,X_test_95,Y_train_95,Y_test_95 = remove_outliers(X_train,Y_train,X_test,Y_test,95,5)

In [295]:
rgsrr = fit_rf(X_train_95,Y_train_95,X_test_95,Y_test_95,150,10,2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


****************************************************************************************************
TRAIN ERROR
MAE: 24.14900249655127
MSE: 3335.4342492220317
RMSE: 57.75321851829586
****************************************************************************************************
TEST ERROR
MAE: 34.11016560850274
MSE: 5442.951493070552
RMSE: 73.77636134339069
****************************************************************************************************
Pred:  845.8990991779648  True:  855.0
Pred:  842.4878145148359  True:  850.0
Pred:  841.9383655033265  True:  840.0
Pred:  840.0811006650764  True:  830.0
Pred:  837.761856060525  True:  810.0
Pred:  826.6501695510673  True:  800.0
Pred:  806.2092078940286  True:  810.0
Pred:  809.5803575971315  True:  825.0
Pred:  829.8193972197344  True:  840.0
Pred:  839.9848159736625  True:  820.0


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


In [296]:
rgsrr.predict(X_test[0].reshape(1,-1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([845.89909918])

In [297]:
X_30 = X_test[:30].copy()

In [298]:
X_30[0]

array([27.25, 78.0, 1.9999847, 79.19499237499998, 11376946.955,
       291.610795, 667.137411941667, 3.7252903, 860.0, 880.0, 940.0,
       1073.5714285714287, 270.0, 263.0, 250.0, 268.2857142857143,
       3.725290299999999, 3.725290299999999, 3.725290299999999,
       99.03063486240862, 0, 0, 0.75819767, -0.773561, -0.8356619,
       -0.014831018, -0.001299955, 0.019895706, 0.0024683475, 0.031677876,
       -0.02698052], dtype=object)

In [348]:
market = 'Agra'
day = 10
month =8



a=[]
index = markets.index(market)
a.append([df_market_embedding['C1'][index],df_market_embedding['C2'][index],df_market_embedding['C3'][index]])
    

b=[]
index = months.index(month)
b.append([df_month_embedding['C1'][index],df_month_embedding['C2'][index],df_month_embedding['C3'][index]])
    
c=[]
index = days.index(day)
c.append([df_day_embedding['C1'][index],df_day_embedding['C2'][index],df_day_embedding['C3'][index]])


In [349]:
check = a[0]+b[0]+c[0]

In [350]:
check

[0.75819767,
 -0.773561,
 -0.8356619,
 -0.35649973,
 0.44832852,
 -0.36272973,
 0.2409199,
 -0.35323268,
 0.1673852]

In [351]:
X_test[0][-9:]

array([0.75819767, -0.773561, -0.8356619, -0.014831018, -0.001299955,
       0.019895706, 0.0024683475, 0.031677876, -0.02698052], dtype=object)

In [352]:
ar = list(X_test[0][-9:])
    
ar

[0.75819767,
 -0.773561,
 -0.8356619,
 -0.014831018,
 -0.001299955,
 0.019895706,
 0.0024683475,
 0.031677876,
 -0.02698052]

In [354]:
for i in range(len(X_test)):
    ar = list(X_test[i][-9:])
    
    if ar == check:
        print('Yes')
        break

Yes


In [357]:
X_test[221]

array([27.25, 78.0, 1.9999847, 88.48336395833333, 4378087.36,
       301.4926183333333, 406908.96039045433, 47765.672, 1850.0, 1800.0,
       1610.0, 1771.4285714285713, 178.0, 215.0, 222.5,
       248.85714285714286, 599239.3336458334, 413763.64594959584,
       904429.776, 392915.7809449626, 0, 0, 0.75819767, -0.773561,
       -0.8356619, -0.35649973, 0.44832852, -0.36272973, 0.2409199,
       -0.35323268, 0.1673852], dtype=object)

In [358]:
rgsrr.predict(X_test[221].reshape(1,-1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([1854.37761241])

In [377]:
rgsrr.predict(X_test[224].reshape(1,-1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([2171.5964812])

In [364]:
X_test[1][9]

860.0

In [359]:
Y_test[221]

1880.0

In [378]:
index = 221
X_test_30 = X_test[index:index+30].copy()
p1 = rgsrr.predict(X_test_30[0].reshape(1,-1))
X_test_30[1][8] = p1
p2 = rgsrr.predict(X_test_30[1].reshape(1,-1))

X_test_30[2][8] = p2
X_test_30[2][9] = p1
p3 = rgsrr.predict(X_test_30[2].reshape(1,-1))

X_test_30[3][8] = p3
X_test_30[3][9] = p2
X_test_30[3][10] =p1
p4 = rgsrr.predict(X_test_30[3].reshape(1,-1))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [375]:
p1,p2,p3

(array([1854.37761241]), array([1855.94306969]), array([1855.62290008]))

In [379]:
p4

array([1855.62290008])

In [382]:
y_pred__ = rgsrr.predict(X_test[index:index+30])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [384]:
y_pred__

array([1854.37761241, 1855.94306969, 1855.62290008, 1855.62290008,
       2174.1011729 , 2175.92745422, 2291.0981746 , 2421.92200301,
       2423.81218994, 2452.14734264, 2407.76925412, 2322.29777538,
       2277.51231716, 2271.81914895, 2013.48907746, 2177.21615685,
       2023.55861591, 2025.35043591, 2023.55861591, 2021.61873358,
       2015.84002703, 2015.84002703, 1952.44073865, 1912.09201806,
       1889.49714726, 1906.07182586, 1908.26833757, 1869.17661333,
       1906.07182586, 1881.24433143])

In [387]:
importances = rgsrr.feature_importances_
importances = np.multiply(importances,10000000)

for i in range(len(importances)):
    print('i: ',i,' importance: ',importances[i])

i:  0  importance:  239.72731713530308
i:  1  importance:  297.69353083990757
i:  2  importance:  5.077233550124545
i:  3  importance:  790.8058989065858
i:  4  importance:  454.61244803176237
i:  5  importance:  869.119592226703
i:  6  importance:  288.1862462195006
i:  7  importance:  289.9099431070168
i:  8  importance:  9783672.619702505
i:  9  importance:  185435.66867430267
i:  10  importance:  20682.04333925503
i:  11  importance:  894.2512982929705
i:  12  importance:  944.7072092321634
i:  13  importance:  495.6097462929155
i:  14  importance:  447.0505127823471
i:  15  importance:  360.72397885463573
i:  16  importance:  359.4160873876715
i:  17  importance:  192.71896572182868
i:  18  importance:  122.8670957756747
i:  19  importance:  663.952856813671
i:  20  importance:  12.59233635954512
i:  21  importance:  9.351172127554424
i:  22  importance:  224.9918196686175
i:  23  importance:  198.56193183977524
i:  24  importance:  241.67684767599505
i:  25  importance:  397.9828